# Template Library Basics

This template intentionally ships only workshop-agnostic helpers in `simulated_city`:

- Load configuration from `config.yaml` + optional `.env` (`simulated_city.config`)
- Build MQTT topics and connect/publish (`simulated_city.mqtt`)

The actual city simulation is an exercise (see `docs/exercises.md`).

In [4]:
# Imports used in this notebook.
import json

from simulated_city.config import load_config
from simulated_city.mqtt import publish_json_checked, topic

In [5]:
# Safety switch: publishing sends a real MQTT message.
# Keep this False unless you really want to publish.
ENABLE_PUBLISH = False

In [6]:
# Load settings from config.yaml (and .env for secrets, if present).
cfg = load_config()

In [7]:
# Build an example topic + JSON payload (no network calls yet).
events_topic = topic(cfg.mqtt, "events/demo")
payload = json.dumps({"hello": "humtek"})

events_topic, payload

('simulated-city/events/demo', '{"hello": "humtek"}')

In [8]:
# Print where the notebook would publish (helps debugging).
print("MQTT broker:", f"{cfg.mqtt.host}:{cfg.mqtt.port}", "tls=", cfg.mqtt.tls)
print("Base topic:", cfg.mqtt.base_topic)
print("Example publish topic:", events_topic)

MQTT broker: c451c402b7fb41b399936cd5727a1d3f.s1.eu.hivemq.cloud:8883 tls= True
Base topic: simulated-city
Example publish topic: simulated-city/events/demo


## Optional: connect and publish

This section publishes **one** MQTT message so you can confirm your broker credentials work.

- Cell 3: set `ENABLE_PUBLISH = True`
- Cell 8: does the actual publish + prints what happened

Tip: in many broker dashboards you’ll want to subscribe to `simulated-city/#` (or your configured base topic + `/#`) to see messages.

In [9]:
# If enabled, publish ONE message and (optionally) verify it by self-subscribing.
if not ENABLE_PUBLISH:
    print("Skipping MQTT publish (ENABLE_PUBLISH is False).")
    print("To enable: set ENABLE_PUBLISH = True in Cell 3.")
else:
    result = publish_json_checked(
        cfg.mqtt,
        topic=events_topic,
        payload=payload,
        client_id_suffix="notebook",
        wait_timeout_s=8.0,
    )
    print(result)
    if result.error:
        print("Issue:", result.error)

Skipping MQTT publish (ENABLE_PUBLISH is False).
To enable: set ENABLE_PUBLISH = True in Cell 3.


## CRS transforms (WGS84 ↔ EPSG:25832)

Most real-world locations are given as **WGS84** latitude/longitude (EPSG:4326).
For distance-based math (meters), it’s often easier to work in **UTM zone 32N** (EPSG:25832) in Denmark.

The next cell uses an approximate point near Copenhagen City Hall and converts:
1) WGS84 (lat/lon) → EPSG:25832 (Easting/Northing in meters)
2) EPSG:25832 → back to WGS84 (to show the round-trip error is tiny)

In [10]:
# Convert a real-world WGS84 point (lat/lon) to UTM32 meters (EPSG:25832) and back.
#
# This cell shows TWO ways to do it:
# 1) The general `transform_xy(...)` function (more flexible, but you must remember axis order)
# 2) The convenience helpers `wgs2utm(...)` / `utm2wgs(...)` (recommended for beginners)

from simulated_city.geo import EPSG_25832, transform_xy, utm2wgs, wgs2utm

# Approximate point near Copenhagen City Hall (Rådhuspladsen).
lat, lon = 55.6761, 12.5683
print("Start (WGS84 lat, lon):", (lat, lon))

try:
    print("\n--- Version A: transform_xy (note axis order) ---")
    # Note: transforms use (x, y) = (lon, lat) when converting to/from EPSG:4326.
    e_a, n_a = transform_xy(lon, lat, from_crs="EPSG:4326", to_crs=EPSG_25832)
    print("To EPSG:25832 (E, N) [m]:", (e_a, n_a))

    lon_back_a, lat_back_a = transform_xy(e_a, n_a, from_crs=EPSG_25832, to_crs="EPSG:4326")
    print("Back to WGS84 (lat, lon):", (lat_back_a, lon_back_a))

    print("\n--- Version B: convenience helpers (beginner-friendly) ---")
    e_b, n_b = wgs2utm(lat, lon)
    print("To EPSG:25832 (E, N) [m]:", (e_b, n_b))

    lat_back_b, lon_back_b = utm2wgs(e_b, n_b)
    print("Back to WGS84 (lat, lon):", (lat_back_b, lon_back_b))

    print("\n--- Compare results (A vs B) ---")
    print("ΔE (m):", e_b - e_a)
    print("ΔN (m):", n_b - n_a)
    print("Round-trip error via helpers (degrees):", (lat_back_b - lat, lon_back_b - lon))
except ModuleNotFoundError as e:
    print(str(e))
    print("Install with: pip install -e \".[geo]\"  (or: pip install pyproj)")

Start (WGS84 lat, lon): (55.6761, 12.5683)

--- Version A: transform_xy (note axis order) ---
To EPSG:25832 (E, N) [m]: (724351.928637642, 6175804.02212861)
Back to WGS84 (lat, lon): (55.67609999999999, 12.568299999999999)

--- Version B: convenience helpers (beginner-friendly) ---
To EPSG:25832 (E, N) [m]: (724351.928637642, 6175804.02212861)
Back to WGS84 (lat, lon): (55.67609999999999, 12.568299999999999)

--- Compare results (A vs B) ---
ΔE (m): 0.0
ΔN (m): 0.0
Round-trip error via helpers (degrees): (-7.105427357601002e-15, -1.7763568394002505e-15)


## Web map example (Copenhagen City Hall)

This section shows a point "in front of" Copenhagen City Hall on a web map.
We start with an approximate WGS84 lat/lon for the City Hall area, then compute 3857 + 25832 for display.

Note: Folium renders as a large HTML/JS output. VS Code can be slow/hang when rendering it inline.
Cell 12 only builds the map object `m`. Cell 13 lets you choose: inline display (JupyterLab) or save HTML (VS Code).

Folium is optional; if it’s not installed, the cell prints install instructions.

In [1]:
# Approximate point near Copenhagen City Hall (Rådhuspladsen).
# If you need an exact surveyed point later, replace these numbers.
lat, lon = 55.6761, 12.5683

# For the web map we don't need any CRS transforms: folium expects WGS84 (lat/lon).
try:
    import folium  # type: ignore
except ModuleNotFoundError:
    folium = None

if folium is None:
    print("folium is not installed.")
    print("Install with: pip install -e \".[notebooks]\"  (or: pip install folium)")
else:
    m = folium.Map(location=[lat, lon], zoom_start=18, tiles="OpenStreetMap")
    popup = folium.Popup(f"WGS84 (lat, lon): {lat:.6f}, {lon:.6f}", max_width=300)
    folium.Marker([lat, lon], popup=popup, tooltip="Copenhagen City Hall (approx)").add_to(m)

In [2]:
from pathlib import Path

# Choose how to view the map:
# - JupyterLab: set INLINE_FOLIUM=True for inline rendering
# - VS Code: keep INLINE_FOLIUM=False and open the saved HTML file
INLINE_FOLIUM = True

if "m" not in globals():
    print("No map object `m` was created (folium may be missing).")
elif INLINE_FOLIUM:
    # Works even if this cell has other output.
    from IPython.display import display  # type: ignore
    display(m)
else:
    map_html_path = Path("copenhagen_city_hall_map.html").resolve()
    m.save(str(map_html_path))
    print("Saved map HTML:", map_html_path)
    print("Open it in your browser (file URL):", map_html_path.as_uri())

In [ ]:
# Intentionally left blank.
# (Avoid rendering the Folium map inline by accident.)